In [2]:
import pandas as pd

In [9]:
colnames=['output','text']
df = pd.read_csv('all-data.csv', names=colnames, header=None)

First, we download our dataset. Data extracted from: https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news

In [30]:
df.head()

,output,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [31]:
list_output = df.output.unique()
list_output

array(['neutral', 'negative', 'positive'], dtype=object)

In [32]:
frequency_output = [(df.output[df['output'] == answer].count(),answer) for answer in list_output]
pd.DataFrame(frequency_output)

,0,1
0,2879,neutral
1,604,negative
2,1363,positive


As seen, there is an imbalance between all three outputs